# Klassifikation mit KNN

Nach Split:
1. Normalisieren der Merkmale
  - euklidischer Abstand



## Bibliotheken Importieren

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import GridSearchCV !! muss auskommentiert sein für gpu grid search
from sklearn.metrics import classification_report, confusion_matrix
# für GPU ausführung:
import cudf
import cupy as cp
import gc
from cuml.neighbors import KNeighborsClassifier as cuKNN
from cuml.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from cuml.preprocessing import StandardScaler as cuStandardScaler
from sklearn.preprocessing import LabelEncoder
import itertools
from cuml import metrics

## Daten einlesen


In [ ]:
meta_train = pd.read_csv("/content/drive/MyDrive/BV-Projekt-SS25/y_train.csv")
meta_val = pd.read_csv("/content/drive/MyDrive/BV-Projekt-SS25/y_val.csv")
meta_test = pd.read_csv("/content/drive/MyDrive/BV-Projekt-SS25/y_test.csv")

embeddings_train =  np.load("/content/drive/MyDrive/BV-Projekt-SS25/emb_train.npy")  # shape (N, d)
embeddings_val =  np.load("/content/drive/MyDrive/BV-Projekt-SS25/emb_val.npy")  # shape (N, d)
embeddings_test =  np.load("/content/drive/MyDrive/BV-Projekt-SS25/emb_test.npy")  # shape (N, d)

In [ ]:
for i in range(embeddings_train.shape[1]):
  meta_train[f"feat_{i}"] = embeddings_train[:, i]


for i in range(embeddings_val.shape[1]):
  meta_val[f"feat_{i}"] = embeddings_val[:, i]


for i in range(embeddings_test.shape[1]):
  meta_test[f"feat_{i}"] = embeddings_test[:, i]

/tmp/ipython-input-92549215.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  meta_train[f"feat_{i}"] = embeddings_train[:, i]
/tmp/ipython-input-92549215.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  meta_train[f"feat_{i}"] = embeddings_train[:, i]
/tmp/ipython-input-92549215.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented fr

In [ ]:
print(meta_train.head())
print(meta_train.columns)


  country    feat_0    feat_1    feat_2    feat_3    feat_4    feat_5  \
0      GR -0.195068  0.041168 -0.212646  0.097290  0.418945  0.032288   
1      GB  0.006927 -0.064758  0.075317  0.043732  0.316650 -0.229126   
2      SE -0.173340 -0.513184 -0.081177  0.295166  0.851074  0.052002   
3      GB -0.632812  0.166138 -0.205200  0.270508  0.679688 -0.019745   
4      PL -0.482910 -0.173340 -0.145874 -0.086060  0.164429 -0.163818   

     feat_6    feat_7    feat_8  ...  feat_502  feat_503  feat_504  feat_505  \
0  0.138672 -0.250977  0.017746  ... -0.184082 -0.054413  0.334717  0.586426   
1  0.511230  0.504883 -0.346436  ... -0.014984  0.486572  0.256836  0.000014   
2  0.339844  0.438965 -0.119934  ...  0.060028  0.690918  0.455811  0.250488   
3  0.090088  0.180786  0.039337  ...  0.042999  0.265381  0.005344 -0.041931   
4  0.012802  0.289551 -0.009872  ...  0.285156  0.043243  0.248657  0.475830   

   feat_506  feat_507  feat_508  feat_509  feat_510  feat_511  
0 -0.114502  0.2

## Arrays erstellen

In [ ]:
# Embeddings direkt verwenden (und gleich in float32 casten, um den Speicher zu halbieren)

# Trainingsdaten
X_train = embeddings_train.astype(np.float32)
y_train = meta_train["country"].values

# Validierungsdaten
X_val = embeddings_val.astype(np.float32)
y_val = meta_val["country"].values

# Testdaten
X_test = embeddings_test.astype(np.float32)
y_test = meta_test["country"].values


# Freigeben ungenutzer objekte -> mehr ram
del meta_train, meta_val, meta_test
del embeddings_train, embeddings_val, embeddings_test


Für GPU Anwendung brauchen wir Umwandlung zu CuPy. CuPy kann nur nummerische DTypes auf GPU schieben, weshalb Label-Encoding zu ganzzahligem NumPy Array umgewandelt werden muss. (DE, FR usw zu zahlen)

In [ ]:

# 1. Labels encodieren
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)   # NumPy-Array int64
y_val_enc   = le.transform(y_val)
y_test_enc  = le.transform(y_test)

# 2. In int32 casten (besser zu GPU)
y_train_enc = y_train_enc.astype(np.int32)
y_val_enc   = y_val_enc.astype(np.int32)
y_test_enc  = y_test_enc.astype(np.int32)

## Feature-Skalierung
Wichtig, sonst dominieren Features mit größerer Varianz die Distanzberechnung.

- Jede Feature-Dimension soll Mittelwert 0 und Standardabweichung 1 erhalten.
- Einheitliche Skala -> faire Gewichtung, da auf Abständen basiert

CLIP Vektoren sind auf Einheitslänge skaliert (L2 Norm = 1), sagt aber nichts über Verteilugn der Dimensionen aus.

Durch Standardisierung gehen keine Informationen verloren, es wird nur die Skalierung verändert.



In [ ]:
# CPU
scaler = StandardScaler().fit(X_train)
X_train_s = scaler.fit_transform(X_train)
X_val_s = scaler.transform(X_val)
X_test_s = scaler.transform(X_test)

# Auf GPU geht nicht wegen RAM, deshalb hier mit CPU und danach auf GPU


In [ ]:
# Freigeben ungenutzer objekte -> mehr ram
del X_train, X_val, X_test

In [ ]:
# aus float32-NumPy -> float32-CuPy
X_train_gpu = cp.asarray(X_train_s)
y_train_gpu = cp.asarray(y_train_enc)

X_val_gpu   = cp.asarray(X_val_s)
y_val_gpu   = cp.asarray(y_val_enc)

X_test_gpu  = cp.asarray(X_test_s)
y_test_gpu  = cp.asarray(y_test_enc)

In [ ]:
# GPU
''' geht nicht wegen RAM
scaler = cuStandardScaler() #.fit(X_train_gpu)?? muss noch dazu idk, noch nicht aber würde in meinen augen sinn machen
X_train_gpu_s = scaler.fit_transform(X_train_gpu)
X_val_gpu_s   = scaler.transform(X_val_gpu)
X_test_gpu_s  = scaler.transform(X_test_gpu)
'''

*OPTIONAL NOCH DAZU*
## Dimensionalitätsreduktion - PCA
Bei sehr hohem D (512) kann PCA helfen die Dimensionalität zu mildern.


**Wofür?**

* Rauschreduktion: Unwichtige, sehr schwache Principal Components verwerfen.
*   Rechen­effizienz: Weniger Dimensionen → schnellere KNN-Suche und geringerer Speicher.

**Pro**

* Entschärft „Fluch der Dimensionalität“.
* Entfernt redundante Linearkombinationen.

**Contra**

* Verlust von feinen Unterschieden, wenn zu stark reduziert.
* Komponenten sind lineare Kombinationen – nicht immer interpretierbar.

**Wann ausführen?**

* Vor dem Training, direkt nach Feature-Skalierung.

* Typischerweise am Anfang als Feature-Engineering, *nicht als finetuning*.

**Konkret:** ***PCA auf Trainings-Features fitten (PCA(n_components=0.95) für 95 % Varianz) und dann auf Val/Test transformieren.***

In [ ]:
#from sklearn.decomposition import PCA

# Beispiel: 95 % Varianz behalten
#pca = PCA(n_components=0.95).fit(X_train_s)
#X_train_pca = pca.transform(X_train_s)
#X_val_pca   = pca.transform(X_val_s)
#X_test_pca  = pca.transform(X_test_s)


## Baseline-Modell
Einfaches KNN Modell mit Standard-K = 5.

In [ ]:
# geht nur, wenn oben nicht RAM freigegeben mit entsprechenden parametern
'''
knn = KNeighborsClassifier(n_neighbors=5,   # Standard-K
                           weights="uniform",
                           metric="minkowski", p=2)  # euklidischer Abstand
knn.fit(X_train_s, y_train)

# Validierungs-Auswertung
y_val_pred = knn.predict(X_val_s)
print(classification_report(y_val, y_val_pred))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_val_pred))
'''



---


*MIT CUDA:*



In [ ]:
# Runntime prüfen - ist GPU ausgewählt?
!nvida-smi

/bin/bash: line 1: nvida-smi: command not found


In [ ]:
# 1. Rapids/cuML in Colab installieren
# -------------------------------------
# (Muss nur einmal pro Session laufen)
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/install_rapids.sh stable

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 597, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 597 (delta 128), reused 82 (delta 82), pack-reused 434 (from 3)
Receiving objects: 100% (597/597), 196.59 KiB | 5.96 MiB/s, done.
Resolving deltas: 100% (302/302), done.
bash: rapidsai-csp-utils/colab/install_rapids.sh: No such file or directory


In [ ]:
# 2. Device-Check
# -------------------------

# Bestätigen, dass cuML CUDA verwendet:
print("cuML GPU-Info:", cp.cuda.runtime.getDeviceCount(), "Device(s) verfügbar")

cuML GPU-Info: 1 Device(s) verfügbar


In [ ]:
# 3. KNN-Baseline auf der GPU
# ---------------------------
knn_base = cuKNN(
    n_neighbors=5,
    weights="uniform",           # gleiches Gewicht für alle Nachbarn
    metric="euclidean"           # euklidischer Abstand
)
knn_base.fit(X_train_gpu, y_train_gpu)

# Eval auf Val-Set
val_pred_gpu = knn_base.predict(X_val_gpu)
# in NumPy zurückholen
val_pred = val_pred_gpu.get().astype(int)

# Speicher frei machen
del val_pred_gpu

# Konvertieren von Zahlen zurück zu Labels (encoding)
val_pred = le.inverse_transform(val_pred)


print("=== Validation Classification Report ===")
print(classification_report(y_val, val_pred, zero_division=0))

print("=== Validation Confusion Matrix ===")
print(confusion_matrix(y_val, val_pred))


=== Validation Classification Report ===
              precision    recall  f1-score   support

          AT       0.43      0.61      0.51      4999
          BE       0.51      0.63      0.56      4996
          BG       0.75      0.88      0.81      4767
          CH       0.64      0.65      0.65      5000
          CZ       0.56      0.58      0.57      4998
          DE       0.45      0.51      0.47     10000
          DK       0.55      0.61      0.58      5000
          EE       0.75      0.84      0.79      4992
          ES       0.68      0.78      0.72      9898
          FI       0.76      0.84      0.80      4999
          FR       0.62      0.54      0.58     10000
          GB       0.75      0.84      0.79     10000
          GR       0.81      0.81      0.81      4999
          HR       0.71      0.59      0.64      4995
          HU       0.67      0.63      0.64      4998
          IE       0.83      0.82      0.82      5000
          IT       0.68      0.60      0

## Hyperparameter-Tuning
### Grid Search für Wahl von K
- Kleine K -> Sehr Komplexes Modell & hohe Varianz (Overfitting)
- Große K -> Weniger Komplexes Modell & hoher Bias (Underfitting)

=> Grid Search plus Visualisierung

Testen verschiedener n_neighbors, Gewichtungsschemata und Metriken


In [ ]:
# Speicher frei machen
del val_pred

In [ ]:
# Aufbauen CuDF statt reine CuPy arrays, da sonst grid search nicht geht

feat_cols = [f"feat_{i}" for i in range(X_train_gpu.shape[1])]

gdf_train = cudf.DataFrame(X_train_gpu, columns=feat_cols)
gdf_train["label"] = y_train_gpu

# !! Explizit in CuPy-Arrays umwandeln (kein impliziter NumPy-Fallback mehr)
X_train_cupy = gdf_train[feat_cols].to_cupy()
y_train_cupy = gdf_train["label"].to_cupy()

# Speicher freigeben
del gdf_train, X_train_gpu, y_train_gpu
gc.collect()

# GPU Speicher aufräumen
cp._default_memory_pool.free_all_blocks()
cp._default_pinned_memory_pool.free_all_blocks()

In [ ]:
import itertools
from cuml.neighbors import KNeighborsClassifier as cuKNN
from cuml import metrics


In [ ]:
# 1) Param-Grid
param_grid = {
    "n_neighbors": [3,5,7,9,11],
    "weights":     ["uniform"],
    "metric":      ["euclidean","manhattan"]
}

best_score = -1.0
best_params = None

# 2) schleife über alle Kombinationen
for k, w, m in itertools.product(param_grid["n_neighbors"],
                                 param_grid["weights"],
                                 param_grid["metric"]):

    # Modell initialisieren & trainieren
    clf = cuKNN(n_neighbors=k, weights=w, metric=m)
    clf.fit(X_train_gpu, y_train_gpu)

    # auf Val-Set vorhersagen & Accuracy berechnen
    y_val_pred_gpu = clf.predict(X_val_gpu)
    acc = metrics.accuracy_score(y_val_gpu, y_val_pred_gpu)

    # bestes Modell merken
    if acc > best_score:
        best_score = acc
        best_params = {"n_neighbors": k, "weights": w, "metric": m}

    # Speicher freigeben für nächsten Durchlauf
    del clf, y_val_pred_gpu
    gc.collect()
    cp._default_memory_pool.free_all_blocks()

print(f" Best Validation Accuracy: {best_score:.4f}")
print(" Beste Parameter:", best_params)

# 3) Finales Modell mit den besten Parametern trainieren
best_knn = cuKNN(**best_params)
best_knn.fit(X_train_gpu, y_train_gpu)

# 4) Validation-Vorhersage holen
val_pred_gpu = best_knn.predict(X_val_gpu)

# 5) In NumPy und Encodings zurückholen
val_pred_enc = val_pred_gpu.get().astype(int)
val_pred_labels = le.inverse_transform(val_pred_enc)

# 6) Metriken ausgeben
print("\n=== Validation Classification Report ===")
print(classification_report(y_val, val_pred_labels, zero_division=0))
print("=== Validation Confusion Matrix ===")
print(confusion_matrix(y_val, val_pred))


# 7) Aufräumen
del val_pred_gpu, val_pred_enc
gc.collect()
cp._default_memory_pool.free_all_blocks()

**!! GPU CuML unterstützt nur unform weights und nicht distance !!**

Deshalb tuning in dem Aspekt weggelassen -> uniform verwendet.

In [ ]:
# GPU
""" geht nicht, weil irgendwie auf falschen GridSearchCV zugreift?? von sklearn ohne GPU??
param_grid = {
    "n_neighbors": [3, 5, 7, 9, 11],
    "weights":     ["uniform", "distance"],
    "metric":      ["euclidean", "manhattan"]
}

grid = GridSearchCV(
    estimator=cuKNN(),
    param_grid=param_grid,
    cv=5,
    scoring="accuracy",
    verbose=2
)
grid.fit(X_train_cupy, y_train_cupy)

print("Beste Params:", grid.best_params_)
best_knn = grid.best_estimator_

# Nochmal Val-Eval mit bestem Modell
gdf_val = cudf.DataFrame(X_val_gpu, columns=feat_cols)
gdf_val["label"] = y_val_gpu # falls später für y_val_gpu benötigt

X_val_cupy = gdf_val[feat_cols].to_cupy()

# Speicher freigeben
del gdf_val, X_val_gpu, y_val_gpu
gc.collect()
cp._default_memory_pool.free_all_blocks()
cp._default_pinned_memory_pool.free_all_blocks()


# 7) Vorhersage & Evaluation
val_pred_gpu = best_knn.predict(X_val_cupy)
val_pred = val_pred_gpu.get().astype(int)
val_pred = le.inverse_transform(val_pred)


print("=== Validation Classification Report ===")
print(classification_report(y_val, val_pred, zero_division=0))
"""

TypeError: Implicit conversion to a NumPy array is not allowed. Please use `.get()` to construct a NumPy array explicitly.

### Lernkurve

In [ ]:
train_fracs = [0.1, 0.3, 0.5, 0.7, 1.0]
train_scores, val_scores = [], []

for frac in train_fracs:
    # Subset-Größe
    n = int(frac * len(X_train_gpu))
    X_sub, y_sub = X_train_gpu[:n], y_train_gpu[:n]

    # Neues KNN mit den besten Parametern
    clf = cuKNN(**grid.best_params_)
    clf.fit(X_sub, y_sub)

    # Accuracy auf Subset (Train) & Val
    tr_acc = metrics.accuracy_score(
        y_sub, clf.predict(X_sub)
    )
    va_acc = metrics.accuracy_score(
        y_val_gpu, clf.predict(X_val_gpu)
    )
    train_scores.append(tr_acc)
    val_scores.append(va_acc)

# Plot
plt.figure(figsize=(8,5))
plt.plot(train_fracs, train_scores, marker='o', label="Train")
plt.plot(train_fracs, val_scores,   marker='o', label="Validation")
plt.xlabel("Anteil des Trainingssets")
plt.ylabel("Accuracy")
plt.title("Lernkurve KNN (GPU mit cuML)")
plt.legend()
plt.grid(True)
plt.show()

## Finale Evaluation auf Test-Set

In [ ]:
y_test_pred = best_knn.predict(X_test_gpu)
print("=== Test-Set Ergebnisse ===")
print(classification_report(y_test_gpu, y_test_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test_gpu, y_test_pred))